In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

def apply_fft(img):
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = np.log(np.abs(fshift) + 1)
    return f, fshift, magnitude_spectrum

def apply_ifft(fshift):
    f_ishift = np.fft.ifftshift(fshift)
    img_back = np.abs(np.fft.ifft2(f_ishift))
    return img_back

def compute_histogram(image, bins=9):
    histogram, _ = np.histogram(image, bins=bins, range=(image.min(), image.max()))
    histogram = histogram / np.sum(histogram)
    return histogram

def extract_statistics(image):
    return {
        'Mean': round(np.mean(image), 3),
        'Std Dev': round(np.std(image), 3),
        'Max': round(np.max(image), 3),
        'Min': round(np.min(image), 3),
        'Median': round(np.median(image), 3)
    }

# Paths for real and fake image directories
real_image_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_real'  # Directory with real images
fake_image_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_fake'  # Directory with fake images
output_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\results\figures\fft'            # Directory to save processed images

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get list of images and limit to 10 each for processing
real_images = sorted([os.path.join(real_image_dir, f) for f in os.listdir(real_image_dir) if f.endswith('.jpg')])[:10]
fake_images = sorted([os.path.join(fake_image_dir, f) for f in os.listdir(fake_image_dir) if f.endswith('.jpg')])[:10]

# Process 10 real and 10 fake images
bins = 128

for i, (real_image_path, fake_image_path) in enumerate(zip(real_images, fake_images)):
    # Load real and fake images as grayscale
    image_0 = cv2.imread(real_image_path, cv2.IMREAD_GRAYSCALE)
    image_1 = cv2.imread(fake_image_path, cv2.IMREAD_GRAYSCALE)
    # Apply FFT and IFFT for real image
    fft_0, fft_shift_0, fft_magnitude_0 = apply_fft(image_0)
    ifft_image_0 = apply_ifft(fft_shift_0)
    histogram_0 = compute_histogram(fft_magnitude_0, bins)
    stats_0 = extract_statistics(fft_magnitude_0)

    # Apply FFT and IFFT for fake image
    fft_1, fft_shift_1, fft_magnitude_1 = apply_fft(image_1)
    ifft_image_1 = apply_ifft(fft_shift_1)
    histogram_1 = compute_histogram(fft_magnitude_1, bins)
    stats_1 = extract_statistics(fft_magnitude_1)

    # Plot and save histograms and FFT results for each image
    fig, axs = plt.subplots(2, 4, figsize=(20, 6))
    
    # Real Image plots
    axs[0][0].imshow(image_0, cmap='gray')
    axs[0][0].set_title(f'Real Image {i+1}')
    axs[0][0].axis('off')

    axs[0][1].imshow(fft_magnitude_0, cmap='gray')
    axs[0][1].set_title('FFT Magnitude Spectrum (Real)')
    axs[0][1].axis('off')

    axs[0][2].imshow(ifft_image_0, cmap='gray')
    axs[0][2].set_title('Reconstructed Image (Real)')
    axs[0][2].axis('off')

    axs[0][3].bar(range(bins), histogram_0, color='blue', alpha=0.7)
    axs[0][3].set_title(f'Histogram (Real) - {bins} Bins')
    axs[0][3].set_xlabel('Bins')
    axs[0][3].set_ylabel('Frequency')

    # Fake Image plots
    axs[1][0].imshow(image_1, cmap='gray')
    axs[1][0].set_title(f'Fake Image {i+1}')
    axs[1][0].axis('off')

    axs[1][1].imshow(fft_magnitude_1, cmap='gray')
    axs[1][1].set_title('FFT Magnitude Spectrum (Fake)')
    axs[1][1].axis('off')

    axs[1][2].imshow(ifft_image_1, cmap='gray')
    axs[1][2].set_title('Reconstructed Image (Fake)')
    axs[1][2].axis('off')

    axs[1][3].bar(range(bins), histogram_1, color='red', alpha=0.7)
    axs[1][3].set_title(f'Histogram (Fake) - {bins} Bins')
    axs[1][3].set_xlabel('Bins')
    axs[1][3].set_ylabel('Frequency')

    # Display statistics below the images
    axs[0][0].text(0.5, -0.3, f'Statistics (Real): {stats_0}', 
                   transform=axs[0][0].transAxes, fontsize=10, ha='center')
    axs[1][0].text(0.5, -0.3, f'Statistics (Fake): {stats_1}', 
                   transform=axs[1][0].transAxes, fontsize=10, ha='center')

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'comparison_image_{i+1}.jpg'))
    plt.close(fig)


In [ ]:
import os
from skimage.feature import local_binary_pattern
import numpy as np
import cv2
import matplotlib.pyplot as plt

def apply_lbp(img, radius=1, n_points=8):
    """
    Apply Local Binary Pattern (LBP) to detect texture inconsistencies in the images.
    
    Args:
        img (np.ndarray): Image to process.
        radius (int): Radius of the LBP pattern.
        n_points (int): Number of points considered in the LBP pattern.
        
    Returns:
        np.ndarray: LBP-processed image.
    """
    lbp = local_binary_pattern(img, n_points, radius, method='uniform')
    return lbp

def compute_histogram(image, bins):
    histogram, _ = np.histogram(image, bins=bins, density=True)
    return histogram

def extract_statistics(image):
    return {
        'Mean': round(np.mean(image), 3),
        'Std Dev': round(np.std(image), 3),
        'Max': round(np.max(image), 3),
        'Min': round(np.min(image), 3),
        'Median': round(np.median(image), 3)
    }

# Paths for real and fake image directories
real_image_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_real'  # Directory with real images
fake_image_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_fake'  # Directory with fake images
output_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\results\figures\lbp'            # Directory to save processed images

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get list of images and limit to 10 each for processing
real_images = sorted([os.path.join(real_image_dir, f) for f in os.listdir(real_image_dir) if f.endswith('.jpg')])[:10]
fake_images = sorted([os.path.join(fake_image_dir, f) for f in os.listdir(fake_image_dir) if f.endswith('.jpg')])[:10]

for i, (real_image_path, fake_image_path) in enumerate(zip(real_images, fake_images)):
    # Load real and fake images as grayscale
    image_0 = cv2.imread(real_image_path, cv2.IMREAD_GRAYSCALE)
    image_1 = cv2.imread(fake_image_path, cv2.IMREAD_GRAYSCALE)

    # Apply LBP
    n_points = 8
    radius = 1
    lbp_0 = apply_lbp(image_0, radius, n_points)
    lbp_1 = apply_lbp(image_1, radius, n_points)

    # Compute histograms for LBP images
    bins = range(0, n_points + 3)
    lbp_hist_0 = compute_histogram(lbp_0, bins)
    lbp_hist_1 = compute_histogram(lbp_1, bins)

    # Set up the plots
    fig, axs = plt.subplots(2, 3, figsize=(15, 8))

    # Original Image (Real)
    axs[0, 0].imshow(image_0, cmap='gray')
    axs[0, 0].set_title('Original Real Image')
    axs[0, 0].axis('off')

    # LBP Image (Real)
    axs[0, 1].imshow(lbp_0, cmap='gray')
    axs[0, 1].set_title('LBP Image (Real)')
    axs[0, 1].axis('off')
    axs[0, 1].text(0.5, -0.2, f'Statistics: {extract_statistics(lbp_0)}', transform=axs[0, 1].transAxes, fontsize=12, ha='center')

    # Histogram of LBP (Real)
    axs[0, 2].bar(bins[:-1], lbp_hist_0, width=0.5, color='gray', alpha=0.7)
    axs[0, 2].set_title('LBP Histogram (Real)')
    axs[0, 2].set_xlabel('LBP Value')
    axs[0, 2].set_ylabel('Normalized Frequency')

    # Original Image (Fake)
    axs[1, 0].imshow(image_1, cmap='gray')
    axs[1, 0].set_title('Original Fake Image')
    axs[1, 0].axis('off')

    # LBP Image (Fake)
    axs[1, 1].imshow(lbp_1, cmap='gray')
    axs[1, 1].set_title('LBP Image (Fake)')
    axs[1, 1].axis('off')
    axs[1, 1].text(0.5, -0.2, f'Statistics: {extract_statistics(lbp_1)}', transform=axs[1, 1].transAxes, fontsize=12, ha='center')

    # Histogram of LBP (Fake)
    axs[1, 2].bar(bins[:-1], lbp_hist_1, width=0.5, color='gray', alpha=0.7)
    axs[1, 2].set_title('LBP Histogram (Fake)')
    axs[1, 2].set_xlabel('LBP Value')
    axs[1, 2].set_ylabel('Normalized Frequency')

    # Save the figure
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'lbp_comparison_{i+1}.png'))
    plt.close(fig)


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Paths for real and fake image directories
real_image_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_real'
fake_image_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\data\raw\real_and_fake_face_detection\real_and_fake_face\training_fake'
output_dir = r'C:\Users\tanbe\OneDrive\Desktop\Works\HWUM\FYP\fakefacedetect\results\figures\sobel'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get list of images and limit to 10 each for processing
real_images = sorted([os.path.join(real_image_dir, f) for f in os.listdir(real_image_dir) if f.endswith('.jpg')])[:10]
fake_images = sorted([os.path.join(fake_image_dir, f) for f in os.listdir(fake_image_dir) if f.endswith('.jpg')])[:10]

# Process each real and fake image
for i, (real_image_path, fake_image_path) in enumerate(zip(real_images, fake_images)):
    # Load real and fake images as grayscale
    image_real = cv2.imread(real_image_path, cv2.IMREAD_GRAYSCALE)
    image_fake = cv2.imread(fake_image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply Sobel filter for real image
    sobel_x_real = cv2.Sobel(image_real, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y_real = cv2.Sobel(image_real, cv2.CV_64F, 0, 1, ksize=3)
    sobel_magnitude_real = np.sqrt(sobel_x_real**2 + sobel_y_real**2)
    
    # Calculate histogram for Sobel magnitude (real)
    hist_real, bin_edges_real = np.histogram(sobel_magnitude_real, bins=9, range=(0, np.max(sobel_magnitude_real)))
    hist_real = hist_real / np.sum(hist_real)  # Normalize histogram
    
    # Apply Sobel filter for fake image
    sobel_x_fake = cv2.Sobel(image_fake, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y_fake = cv2.Sobel(image_fake, cv2.CV_64F, 0, 1, ksize=3)
    sobel_magnitude_fake = np.sqrt(sobel_x_fake**2 + sobel_y_fake**2)
    
    # Calculate histogram for Sobel magnitude (fake)
    hist_fake, bin_edges_fake = np.histogram(sobel_magnitude_fake, bins=9, range=(0, np.max(sobel_magnitude_fake)))
    hist_fake = hist_fake / np.sum(hist_fake)  # Normalize histogram

    # Plot and save histograms
    fig, axs = plt.subplots(2, 3, figsize=(15, 8))
    
    # Original Image (Real)
    axs[0, 0].imshow(image_0, cmap='gray')
    axs[0, 0].set_title('Original Real Image')
    axs[0, 0].axis('off')

    # LBP Image (Real)
    axs[0, 1].imshow(sobel_magnitude_real, cmap='gray')
    axs[0, 1].set_title('LBP Image (Real)')
    axs[0, 1].axis('off')
    axs[0, 1].text(0.5, -0.2, f'Statistics: {extract_statistics(lbp_0)}', transform=axs[0, 1].transAxes, fontsize=12, ha='center')

    # Histogram for real image
    axs[0,2].plot(bin_edges_real[:-1], hist_real, color='blue')
    axs[0,2].set_title(f"Sobel Magnitude Histogram (Real Image {i+1})")
    axs[0,2].set_xlabel("Gradient Magnitude")
    axs[0,2].set_ylabel("Frequency")
    
    # Histogram for fake image
    # Original Image (Real)
    axs[1, 0].imshow(image_1, cmap='gray')
    axs[1, 0].set_title('Original Real Image')
    axs[1, 0].axis('off')

    # LBP Image (Real)
    axs[1, 1].imshow(sobel_magnitude_fake, cmap='gray')
    axs[1, 1].set_title('LBP Image (Real)')
    axs[1, 1].axis('off')
    axs[1, 1].text(0.5, -0.2, f'Statistics: {extract_statistics(lbp_0)}', transform=axs[0, 1].transAxes, fontsize=12, ha='center')

    axs[1,2].plot(bin_edges_fake[:-1], hist_fake, color='red')
    axs[1,2].set_title(f"Sobel Magnitude Histogram (Fake Image {i+1})")
    axs[1,2].set_xlabel("Gradient Magnitude")
    axs[1,2].set_ylabel("Frequency")
    
    # Save the histogram plot
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'sobel_histogram_{i+1}.png'))
    plt.close(fig)

    # Save Sobel magnitude images for visualization if needed
    cv2.imwrite(os.path.join(output_dir, f'real_sobel_{i+1}.jpg'), sobel_magnitude_real)
    cv2.imwrite(os.path.join(output_dir, f'fake_sobel_{i+1}.jpg'), sobel_magnitude_fake)

    # Print histogram values for debugging
    print(f"Histogram for Real Image {i+1}: {hist_real}")
    print(f"Histogram for Fake Image {i+1}: {hist_fake}")
